In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-09-26 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# SWNU Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# w=5

In [22]:
size = 5

## UMAP

In [11]:
(
    swnu_network_umap_kfold_5,
    best_swnu_network_umap_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [12]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.636207   
                                                       0.0005         0.641124   
                                                       0.0010         0.636015   
                                          0.2          0.0001         0.633908   
                                                       0.0005         0.637037   
                                                       0.0010         0.633269   
                           (512, 512)     0.1          0.0001         0.629119   
                                                       0.0005         0.634547   
                                                       0.0010         0.633014   
                                          0.2          0.0001         0.632375   
                                                       0.0005         0.641124   
                                                       0.0010         0.635185   
           (12,)           (256, 256)     0.1          0.0001         0.635121   
                                                       0.0005         0.637229   
                                                       0.0010         0.633014   
                                          0.2          0.0001         0.636015   
                                                       0.0005         0.631801   
                                                       0.0010         0.629821   
                           (512, 512)     0.1          0.0001         0.639017   
                                                       0.0005         0.639144   
                                                       0.0010         0.632759   
                                          0.2          0.0001         0.636398   
                                                       0.0005         0.635249   
                                                       0.0010         0.634100   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.517120   
                                                       0.0005         0.519248   
                                                       0.0010         0.514764   
                                          0.2          0.0001         0.514416   
                                                       0.0005         0.520095   
                                                       0.0010         0.512613   
                           (512, 512)     0.1          0.0001         0.512011   
                                                       0.0005         0.516610   
                                                       0.0010         0.514058   
                                          0.2          0.0001         0.513389   
                                                       0.0005         0.519894   
                                                       0.0010         0.517250   
           (12,)           (256, 256)     0.1          0.0001         0.513103   
                                                       0.0005         0.521177   
                                                       0.0010         0.515539   
                                          0.2          0.0001         0.512951   
                                                       0.0005         0.517404   
                                                       0.0010         0.514111   
                           (512, 512)     0.1          0.0001         0.515458   
                                                       0.0005         0.519821   
                                                       0.0010         0.514429   
                                          0.2      

In [13]:
best_swnu_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.637165,0.519572,"[0.7667568379921852, 0.44970414201183434, 0.34...",0.512834,"[0.7981852315394243, 0.4610051993067591, 0.279...",0.539491,"[0.7377096587622903, 0.4389438943894389, 0.441...",None,0.666024,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.635057,0.524985,"[0.7604435667628741, 0.45958145260566263, 0.35...",0.515919,"[0.80096, 0.45714285714285713, 0.2896551724137...",0.548019,"[0.7238288027761712, 0.46204620462046203, 0.45...",None,0.657354,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.633525,0.514677,"[0.761904761904762, 0.43298969072164945, 0.349...",0.508335,"[0.7847946045370938, 0.45161290322580644, 0.28...",0.532657,"[0.7403123192596877, 0.4158415841584158, 0.441...",None,0.666185,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swnu_network_umap_kfold_5["f1"].mean()

0.5197446090694444

In [15]:
best_swnu_network_umap_kfold_5["precision"].mean()

0.512362765312797

In [16]:
best_swnu_network_umap_kfold_5["recall"].mean()

0.5400556273094054

In [17]:
np.stack(best_swnu_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.76303506, 0.4474251 , 0.34877368])

In [18]:
np.stack(best_swnu_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.79464661, 0.45658699, 0.2858547 ])

In [19]:
np.stack(best_swnu_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.73395026, 0.43894389, 0.44727273])

## GRP

In [23]:
(
    swnu_network_grp_kfold_5,
    best_swnu_network_grp_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_5_kfold_best_model.csv


In [24]:
swnu_network_grp_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/543207788.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.637931   
                                                       0.0005         0.634674   
                                                       0.0010         0.637867   
                                          0.2          0.0001         0.636782   
                                                       0.0005         0.634100   
                                                       0.0010         0.631737   
                           (512, 512)     0.1          0.0001         0.635824   
                                                       0.0005         0.638314   
                                                       0.0010         0.630140   
                                          0.2          0.0001         0.635441   
                                                       0.0005         0.636718   
                                                       0.0010         0.633142   
           (12,)           (256, 256)     0.1          0.0001         0.633333   
                                                       0.0005         0.629949   
                                                       0.0010         0.630140   
                                          0.2          0.0001         0.631354   
                                                       0.0005         0.629310   
                                                       0.0010         0.635504   
                           (512, 512)     0.1          0.0001         0.631673   
                                                       0.0005         0.642082   
                                                       0.0010         0.629821   
                                          0.2          0.0001         0.630971   
                                                       0.0005         0.639208   
                                                       0.0010         0.633014   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.519559   
                                                       0.0005         0.520334   
                                                       0.0010         0.522015   
                                          0.2          0.0001         0.518263   
                                                       0.0005         0.521276   
                                                       0.0010         0.520523   
                           (512, 512)     0.1          0.0001         0.518346   
                                                       0.0005         0.520946   
                                                       0.0010         0.515548   
                                          0.2          0.0001         0.520424   
                                                       0.0005         0.521867   
                                                       0.0010         0.516289   
           (12,)           (256, 256)     0.1          0.0001         0.515543   
                                                       0.0005         0.518834   
                                                       0.0010         0.517478   
                                          0.2          0.0001         0.515484   
                                                       0.0005         0.518513   
                                                       0.0010         0.521285   
                           (512, 512)     0.1          0.0001         0.514363   
                                                       0.0005         0.522722   
                                                       0.0010         0.513611   
                                          0.2      

In [25]:
swnu_network_grp_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/543207788.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.637931   
                                                       0.0005         0.634674   
                                                       0.0010         0.637867   
                                          0.2          0.0001         0.636782   
                                                       0.0005         0.634100   
                                                       0.0010         0.631737   
                           (512, 512)     0.1          0.0001         0.635824   
                                                       0.0005         0.638314   
                                                       0.0010         0.630140   
                                          0.2          0.0001         0.635441   
                                                       0.0005         0.636718   
                                                       0.0010         0.633142   
           (12,)           (256, 256)     0.1          0.0001         0.633333   
                                                       0.0005         0.629949   
                                                       0.0010         0.630140   
                                          0.2          0.0001         0.631354   
                                                       0.0005         0.629310   
                                                       0.0010         0.635504   
                           (512, 512)     0.1          0.0001         0.631673   
                                                       0.0005         0.642082   
                                                       0.0010         0.629821   
                                          0.2          0.0001         0.630971   
                                                       0.0005         0.639208   
                                                       0.0010         0.633014   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.519559   
                                                       0.0005         0.520334   
                                                       0.0010         0.522015   
                                          0.2          0.0001         0.518263   
                                                       0.0005         0.521276   
                                                       0.0010         0.520523   
                           (512, 512)     0.1          0.0001         0.518346   
                                                       0.0005         0.520946   
                                                       0.0010         0.515548   
                                          0.2          0.0001         0.520424   
                                                       0.0005         0.521867   
                                                       0.0010         0.516289   
           (12,)           (256, 256)     0.1          0.0001         0.515543   
                                                       0.0005         0.518834   
                                                       0.0010         0.517478   
                                          0.2          0.0001         0.515484   
                                                       0.0005         0.518513   
                                                       0.0010         0.521285   
                           (512, 512)     0.1          0.0001         0.514363   
                                                       0.0005         0.522722   
                                                       0.0010         0.513611   
                                          0.2      

In [26]:
best_swnu_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.629885,0.512292,"[0.7563527653213753, 0.43732018084669133, 0.34...",0.504387,"[0.7827970297029703, 0.4357084357084357, 0.294...",0.527163,"[0.7316367842683632, 0.4389438943894389, 0.410...",None,0.668112,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.631226,0.516510,"[0.7575757575757577, 0.4415584415584416, 0.350...",0.508643,"[0.7870947630922693, 0.44851063829787235, 0.29...",0.535609,"[0.7301908617698092, 0.43481848184818483, 0.44...",None,0.662492,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.633908,0.514286,"[0.7594074074074074, 0.4338756779307468, 0.349...",0.507182,"[0.778554070473876, 0.4388185654008439, 0.3041...",0.527044,"[0.7411798727588201, 0.429042904290429, 0.4109...",None,0.670199,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [27]:
best_swnu_network_grp_kfold_5["f1"].mean()

0.5143628505176907

In [28]:
best_swnu_network_grp_kfold_5["precision"].mean()

0.5067369839920379

In [29]:
best_swnu_network_grp_kfold_5["recall"].mean()

0.529938795884601

In [30]:
np.stack(best_swnu_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.75777864, 0.43758477, 0.34772514])

In [31]:
np.stack(best_swnu_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78281529, 0.44101255, 0.29638312])

In [32]:
np.stack(best_swnu_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.73433584, 0.43426843, 0.42121212])

# w=11

In [33]:
size = 11

## UMAP

In [34]:
(
    swnu_network_umap_kfold_11,
    best_swnu_network_umap_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [35]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.633653   
                                                       0.0005         0.628225   
                                                       0.0010         0.630013   
                                          0.2          0.0001         0.634227   
                                                       0.0005         0.632248   
                                                       0.0010         0.629310   
                           (512, 512)     0.1          0.0001         0.638633   
                                                       0.0005         0.625223   
                                                       0.0010         0.631481   
                                          0.2          0.0001         0.634994   
                                                       0.0005         0.633142   
                                                       0.0010         0.629757   
           (12,)           (256, 256)     0.1          0.0001         0.633653   
                                                       0.0005         0.629310   
                                                       0.0010         0.629374   
                                          0.2          0.0001         0.635057   
                                                       0.0005         0.626820   
                                                       0.0010         0.626948   
                           (512, 512)     0.1          0.0001         0.632120   
                                                       0.0005         0.626820   
                                                       0.0010         0.634738   
                                          0.2          0.0001         0.631673   
                                                       0.0005         0.631673   
                                                       0.0010         0.630140   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.511955   
                                                       0.0005         0.510760   
                                                       0.0010         0.513454   
                                          0.2          0.0001         0.510878   
                                                       0.0005         0.512480   
                                                       0.0010         0.516421   
                           (512, 512)     0.1          0.0001         0.515204   
                                                       0.0005         0.514040   
                                                       0.0010         0.512974   
                                          0.2          0.0001         0.512026   
                                                       0.0005         0.514013   
                                                       0.0010         0.513605   
           (12,)           (256, 256)     0.1          0.0001         0.510770   
                                                       0.0005         0.509880   
                                                       0.0010         0.517345   
                                          0.2          0.0001         0.509775   
                                                       0.0005         0.509665   
                                                       0.0010         0.515713   
                           (512, 512)     0.1          0.0001         0.509966   
                                                       0.0005         0.513599   
                                                       0.0010         0.513805   
                                          0.2      

In [36]:
best_swnu_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.632375,0.513024,"[0.7635711081202332, 0.4328037784456848, 0.342...",0.507011,"[0.7906472592133788, 0.4512085944494181, 0.279...",0.532589,"[0.738288027761712, 0.4158415841584158, 0.4436...",None,0.659602,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.623755,0.510287,"[0.7518796992481204, 0.42736486486486486, 0.35...",0.502541,"[0.7832080200501254, 0.43771626297577854, 0.28...",0.531666,"[0.7229612492770388, 0.4174917491749175, 0.454...",None,0.662171,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.634291,0.509492,"[0.7609841827768014, 0.42397914856646396, 0.34...",0.504893,"[0.7709198813056379, 0.44770642201834865, 0.29...",0.521011,"[0.7513013302486987, 0.40264026402640263, 0.40...",None,0.668433,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [37]:
best_swnu_network_umap_kfold_11["f1"].mean()

0.5109342557602629

In [38]:
best_swnu_network_umap_kfold_11["precision"].mean()

0.5048147245190061

In [39]:
best_swnu_network_umap_kfold_11["recall"].mean()

0.5284218813244347

In [40]:
np.stack(best_swnu_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75881166, 0.42804926, 0.34594184])

In [41]:
np.stack(best_swnu_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78159172, 0.44554376, 0.28730869])

In [42]:
np.stack(best_swnu_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.73751687, 0.4119912 , 0.43575758])

## GRP

In [43]:
(
    swnu_network_grp_kfold_11,
    best_swnu_network_grp_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_11_kfold_best_model.csv


In [44]:
swnu_network_grp_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/3519552416.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.635951   
                                                       0.0005         0.635888   
                                                       0.0010         0.636271   
                                          0.2          0.0001         0.634483   
                                                       0.0005         0.635568   
                                                       0.0010         0.636462   
                           (512, 512)     0.1          0.0001         0.637995   
                                                       0.0005         0.637676   
                                                       0.0010         0.633078   
                                          0.2          0.0001         0.636079   
                                                       0.0005         0.639911   
                                                       0.0010         0.635441   
           (12,)           (256, 256)     0.1          0.0001         0.635760   
                                                       0.0005         0.630779   
                                                       0.0010         0.631354   
                                          0.2          0.0001         0.636143   
                                                       0.0005         0.626245   
                                                       0.0010         0.632503   
                           (512, 512)     0.1          0.0001         0.641252   
                                                       0.0005         0.636718   
                                                       0.0010         0.626564   
                                          0.2          0.0001         0.639527   
                                                       0.0005         0.642784   
                                                       0.0010         0.629119   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.517082   
                                                       0.0005         0.518681   
                                                       0.0010         0.519882   
                                          0.2          0.0001         0.516197   
                                                       0.0005         0.521268   
                                                       0.0010         0.518802   
                           (512, 512)     0.1          0.0001         0.520074   
                                                       0.0005         0.519678   
                                                       0.0010         0.514910   
                                          0.2          0.0001         0.518820   
                                                       0.0005         0.521451   
                                                       0.0010         0.517726   
           (12,)           (256, 256)     0.1          0.0001         0.520263   
                                                       0.0005         0.517288   
                                                       0.0010         0.518922   
                                          0.2          0.0001         0.518940   
                                                       0.0005         0.515036   
                                                       0.0010         0.519894   
                           (512, 512)     0.1          0.0001         0.519958   
                                                       0.0005         0.520448   
                                                       0.0010         0.511301   
                                          0.2      

In [45]:
swnu_network_grp_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1505186/3519552416.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.635951   
                                                       0.0005         0.635888   
                                                       0.0010         0.636271   
                                          0.2          0.0001         0.634483   
                                                       0.0005         0.635568   
                                                       0.0010         0.636462   
                           (512, 512)     0.1          0.0001         0.637995   
                                                       0.0005         0.637676   
                                                       0.0010         0.633078   
                                          0.2          0.0001         0.636079   
                                                       0.0005         0.639911   
                                                       0.0010         0.635441   
           (12,)           (256, 256)     0.1          0.0001         0.635760   
                                                       0.0005         0.630779   
                                                       0.0010         0.631354   
                                          0.2          0.0001         0.636143   
                                                       0.0005         0.626245   
                                                       0.0010         0.632503   
                           (512, 512)     0.1          0.0001         0.641252   
                                                       0.0005         0.636718   
                                                       0.0010         0.626564   
                                          0.2          0.0001         0.639527   
                                                       0.0005         0.642784   
                                                       0.0010         0.629119   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.517082   
                                                       0.0005         0.518681   
                                                       0.0010         0.519882   
                                          0.2          0.0001         0.516197   
                                                       0.0005         0.521268   
                                                       0.0010         0.518802   
                           (512, 512)     0.1          0.0001         0.520074   
                                                       0.0005         0.519678   
                                                       0.0010         0.514910   
                                          0.2          0.0001         0.518820   
                                                       0.0005         0.521451   
                                                       0.0010         0.517726   
           (12,)           (256, 256)     0.1          0.0001         0.520263   
                                                       0.0005         0.517288   
                                                       0.0010         0.518922   
                                          0.2          0.0001         0.518940   
                                                       0.0005         0.515036   
                                                       0.0010         0.519894   
                           (512, 512)     0.1          0.0001         0.519958   
                                                       0.0005         0.520448   
                                                       0.0010         0.511301   
                                          0.2      

In [46]:
best_swnu_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.633333,0.516964,"[0.7592203898050974, 0.44256348246674726, 0.34...",0.508461,"[0.788293897882939, 0.4326241134751773, 0.3044...",0.531425,"[0.7322151532677849, 0.452970297029703, 0.4090...",None,0.670199,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.649808,0.523870,"[0.7722368037328668, 0.4507042253521127, 0.348...",0.519722,"[0.7788235294117647, 0.46684350132625996, 0.31...",0.531377,"[0.7657605552342395, 0.43564356435643564, 0.39...",None,0.675498,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.640613,0.519040,"[0.7656250000000001, 0.438921651221567, 0.3525...",0.512642,"[0.780817799158148, 0.4483648881239243, 0.3087...",0.530596,"[0.7510121457489879, 0.4298679867986799, 0.410...",None,0.671484,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [47]:
best_swnu_network_grp_kfold_11["f1"].mean()

0.5199579746606222

In [48]:
best_swnu_network_grp_kfold_11["precision"].mean()

0.5136082461799552

In [49]:
best_swnu_network_grp_kfold_11["recall"].mean()

0.5311329972403448

In [50]:
np.stack(best_swnu_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.76569406, 0.44406312, 0.35011674])

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.78264508, 0.4492775 , 0.30890216])

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.74966262, 0.43949395, 0.40424242])

# w=20

In [10]:
size = 20

## UMAP

In [11]:
(
    swnu_network_umap_kfold_20,
    best_swnu_network_umap_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [12]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1432666/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.627650   
                                                       0.0005         0.634994   
                                                       0.0010         0.633078   
                                          0.2          0.0001         0.632631   
                                                       0.0005         0.626054   
                                                       0.0010         0.623946   
                           (512, 512)     0.1          0.0001         0.625734   
                                                       0.0005         0.622542   
                                                       0.0010         0.634930   
                                          0.2          0.0001         0.625862   
                                                       0.0005         0.629566   
                                                       0.0010         0.627395   
           (12,)           (256, 256)     0.1          0.0001         0.628352   
                                                       0.0005         0.629246   
                                                       0.0010         0.628161   
                                          0.2          0.0001         0.627842   
                                                       0.0005         0.640230   
                                                       0.0010         0.626820   
                           (512, 512)     0.1          0.0001         0.628416   
                                                       0.0005         0.626820   
                                                       0.0010         0.626501   
                                          0.2          0.0001         0.626309   
                                                       0.0005         0.636462   
                                                       0.0010         0.629630   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.494926   
                                                       0.0005         0.514723   
                                                       0.0010         0.514975   
                                          0.2          0.0001         0.496811   
                                                       0.0005         0.510704   
                                                       0.0010         0.510259   
                           (512, 512)     0.1          0.0001         0.502020   
                                                       0.0005         0.508704   
                                                       0.0010         0.513492   
                                          0.2          0.0001         0.500076   
                                                       0.0005         0.510827   
                                                       0.0010         0.513285   
           (12,)           (256, 256)     0.1          0.0001         0.500355   
                                                       0.0005         0.508543   
                                                       0.0010         0.512312   
                                          0.2          0.0001         0.500123   
                                                       0.0005         0.514742   
                                                       0.0010         0.514074   
                           (512, 512)     0.1          0.0001         0.500180   
                                                       0.0005         0.509916   
                                                       0.0010         0.508240   
                                          0.2      

In [13]:
best_swnu_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.643870,0.511869,"[0.7680988816951148, 0.45691699604743086, 0.31...",0.509062,"[0.7819053325344517, 0.4385432473444613, 0.306...",0.515405,"[0.7547715442452284, 0.4768976897689769, 0.314...",None,0.671965,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.631609,0.514124,"[0.7606056063558688, 0.4337448559670783, 0.348...",0.505864,"[0.7896047307812014, 0.43267651888341546, 0.29...",0.530705,"[0.7336610757663389, 0.43481848184818483, 0.42...",None,0.667951,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.645211,0.518232,"[0.7710668025032746, 0.42753313381787084, 0.35...",0.513955,"[0.7761500146498681, 0.44365572315882873, 0.32...",0.525591,"[0.7660497397339503, 0.41254125412541254, 0.39...",None,0.678227,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.5147419120105536

In [15]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.509626944353249

In [16]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.5239003802057476

In [17]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.76659043, 0.43939833, 0.33823698])

In [18]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78255336, 0.43829183, 0.30803564])

In [19]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.75149412, 0.44141914, 0.37878788])

## GRP

In [ ]:
(
    swnu_network_grp_kfold_20,
    best_swnu_network_grp_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [ ]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_20

In [ ]:
best_swnu_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## UMAP

In [ ]:
(
    swnu_network_umap_kfold_35,
    best_swnu_network_umap_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

In [ ]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_35

In [ ]:
best_swnu_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    swnu_network_grp_kfold_35,
    best_swnu_network_grp_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

In [ ]:
swnu_network_grp_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_35

In [ ]:
best_swnu_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["recall_scores"]).mean(axis=0)